In [7]:
from yaml import load 
from datetime import datetime, timedelta
import requests
from pprint import pprint
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import httplib2
import argparse
from apiclient import discovery

SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
CLIENT_SECRET_FILE ='client_secret_grants.json'
APPLICATION_NAME ='Google Sheets API Report'

credential_path = 'sheets.googleapis.com-report.json'
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
    print('Storing credentials to ' + credential_path)

In [13]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)

spreadsheetId = '1v_BJppYtDnFTZAIVH_crUMtkbrcShOS1cXCCPYBvb50'
rangeName = 'partners dict!A1:C10'

f = open('config.yaml', 'r')
config = load(f)
token = config['token']
f = open('report_params.yaml', 'r')
params = load(f)
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2018-02-24'
endDate = '2018-03-10'
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

In [14]:
def make_request(**kwargs):
    return requests.get(API_URL, params = kwargs).json()

def json_handler(data):
    report = []
    for line in data['data']:
        dimensions_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append(dimensions_list + metrics_list)
    return report

In [15]:
response = make_request(date1 = startDate, date2 = endDate, dimensions = dimensions, metrics = metrics, 
                    id = counter, sort = sortby, oauth_token = token, accuracy = 1)
data = json_handler(response)
range_data = params['reports']['dau']['range_name']
range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)
body = {'values': data}

In [16]:
service.spreadsheets().values().update( spreadsheetId = spreadsheetId, range = range_name, 
                    valueInputOption = 'USER_ENTERED', body = body ).execute()
new_table = service.spreadsheets().values().get( spreadsheetId = spreadsheetId, range = range_name).execute()

services = new_table.get('values', [])
print(services)

[['2018-02-24', '216', '201', '419', '15,27777778'], ['2018-02-25', '202', '193', '386', '15,34653465'], ['2018-02-26', '279', '264', '440', '13,62007168'], ['2018-02-27', '280', '258', '481', '18,21428571'], ['2018-02-28', '294', '278', '399', '15,98639456'], ['2018-03-01', '293', '272', '430', '15,35836177'], ['2018-03-02', '317', '284', '441', '12,30283912'], ['2018-03-03', '342', '292', '551', '15,78947368'], ['2018-03-04', '265', '233', '429', '19,62264151'], ['2018-03-05', '282', '257', '427', '17,0212766'], ['2018-03-06', '258', '229', '389', '17,82945736'], ['2018-03-07', '203', '185', '318', '12,31527094'], ['2018-03-08', '198', '168', '382', '11,61616162'], ['2018-03-09', '258', '232', '514', '18,21705426'], ['2018-03-10', '221', '198', '429', '20,36199095']]


In [17]:
import pandas as pd
rows = services[1:]
data = pd.DataFrame.from_records(rows)
data

,0,1,2,3,4
0,2018-02-25,202,193,386,"15,34653465"
1,2018-02-26,279,264,440,"13,62007168"
2,2018-02-27,280,258,481,"18,21428571"
3,2018-02-28,294,278,399,"15,98639456"
4,2018-03-01,293,272,430,"15,35836177"
5,2018-03-02,317,284,441,"12,30283912"
6,2018-03-03,342,292,551,"15,78947368"
7,2018-03-04,265,233,429,"19,62264151"
8,2018-03-05,282,257,427,"17,0212766"
9,2018-03-06,258,229,389,"17,82945736"


In [18]:
visits = 0
for visit in data[1]:
    visits += int(visit)
visits

3692